In [7]:
#!/usr/bin/python3
#-*- coding:utf-8 -*-
# https://dev.to/petercour/image-scraping-with-python-330a

import re, os, requests, time, sys
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from PIL import Image
from io import BytesIO
from datetime import datetime
from IPython.display import clear_output


def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1
        
    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)
            
def download_img_src(directory, keyword, url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    i = 0
    
    for raw_img in soup.find_all('img'):
        try:
            link = raw_img.get('src')
            if (not (link == None)):
                final_url = None
                if (link[:4] == 'http'):
                    final_url = link
                else:
                    final_url = url[:len(url)-1] + link
                print(" Getting src from:" + final_url)

                time_part = datetime.now().strftime('%Y%m%d%H%M%S')
                pic = requests.get(final_url, timeout=10)    
                string = directory + '/pic_' + keyword + '_' + time_part + '_' + str(i) + '.gif'
                fp = open(string, 'wb')
                fp.write(pic.content)
                fp.close()
                i += 1
        except:
            print('exception')
            continue

def downloader_recursive(directory, keyword, url):
    update_progress(0.0)
    print("Scanning " + url)
    
    # Scan sub urls
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    bag = []
    bag.append(url)
    
    for href in soup.find_all('a'):
        try:
            jump_link = str(href.get('href'))
            sub_url = None
            if (jump_link[:4] == 'http'):
                sub_url = jump_link
            else:
                sub_url = url + jump_link

            if sub_url not in bag:  
                print("Adding " + sub_url)
                bag.append(sub_url)
        except:
            print('exception')
            continue
    
    # Iterate over all sites
    number_of_elements = len(bag)
    for i in range(number_of_elements):
        print(" Getting src from:" + bag[i])
        download_img_src(directory, keyword, bag[i])
        update_progress(i / number_of_elements)
    update_progress(1.0)
    return bag

# Building classifier for cheking innapropiate photos
downloader_recursive("source", "remeras", 'https://demozoo.org/search/?q=ascii&category=')
print("end")

Progress: [####################] 100.0%
end
